In [1]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
%matplotlib qt

In [3]:
def camera_calib(filenames,nx=9,ny=6):
    objp = np.zeros((ny*nx,3),np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2) #here we created a nice chessboard
    
    imgpoints = []
    objpoints = []

    for fname in filenames:
        img = cv2.imread(fname)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    return (ret, mtx, dist, rvecs, tvecs)

In [4]:
def undistort(img, cam_params):
    ret, mtx, dist, rvecs, tvecs = cam_params
    return cv2.undistort(img, mtx, dist, None, mtx) 

In [63]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [137]:
def convert_to_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0,255)):
    gray = convert_to_gray(img)

    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary_output

def mag_thresh(img, sobel_kernel=3, thresh=(0, 255)):
    # Convert to grayscale
    gray = convert_to_gray(img)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 1
    return binary_output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=9, thresh=(0, np.pi/2)):
    # Grayscale
    gray = convert_to_gray(img)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def Canny_threshold(img, thresh=(0,255)):
    # Grayscale
    gray = convert_to_gray(img)
    binary_output = cv2.Canny(gray,thresh[0],thresh[1])
    binary_output[binary_output==255]=1
    return binary_output
    
def hls_threshold(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # s channel is the best for this threshold
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

def combine_Sobel(img, SobelX=True, SobelY=True, SobelX_params=[], SobelY_params=[]):
    gray = convert_to_gray(img)
    binary_output=np.ones_like(gray)
    
    if(SobelX):
        img, sobel_kernel, thresh = SobelX_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='x', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
    
    if(SobelX):
        img, sobel_kernel, thresh = SobelY_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='y', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
        
    return binary_output    

def combine_MagDir(img, Mag=True, Dir=True, Mag_params=[], Dir_params=[]):
    gray = convert_to_gray(img)
    binary_output=np.ones_like(gray)
    
    if(Mag):
        img, sobel_kernel, thresh = Mag_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='x', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
    
    if(Dir):
        img, sobel_kernel, thresh = Dir_params
        binary_output = binary_output & abs_sobel_thresh(img=img, orient='y', 
                                                         sobel_kernel=sobel_kernel, thresh=thresh)
    
    return binary_output

In [146]:
def combine_threshold(img, Canny=True, Sobel=True, MagDir=True, HLS=True,
                     Canny_params=[], Sobel_params=[], MagDir_params=[], HLS_params=[]):

    #only for resizing
    gray = convert_to_gray(img)  
    binary_output=np.zeros_like(gray)
        
    if(Canny):
        img, thresh = Canny_params
        binary_output = binary_output | Canny_threshold(img, thresh)
        canny_img = Canny_threshold(img,thresh)
        
    if(Sobel):
        img, SobelX, SobelY, SobelX_params, SobelY_params = Sobel_params
        binary_output = binary_output | combine_Sobel(img, SobelX, SobelY, SobelX_params, SobelY_params)
        
    if(MagDir):
        img, Mag, Dir, Mag_params, Dir_params = MagDir_params
        binary_output = binary_output | combine_MagDir(img, Mag, Dir, Mag_params, Dir_params)
        
    if(HLS):
        img, thresh = HLS_params
        binary_output = binary_output | hls_threshold(img,thresh)
        hls_img = hls_threshold(img,thresh)

    return binary_output

In [77]:
def get_perspective_transform(src = np.float32([[373,605], [920,605], [542, 487], [749,487]]),
                             dst = np.float32([[373,605], [920,605], [373, 487], [920,487]])):
    return cv2.getPerspectiveTransform(src,dst)

def perspective_transform(img,M):
    img_size = (img.shape[1], img.shape[0])
    return cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

In [204]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def mask_image(img):
    imshape = img.shape
    left_bottom = (100, imshape[0])
    right_bottom = (1250, imshape[0])
    left_top = (660, 410) 
    right_top = (640, 410)
    in_left_bottom = (310, imshape[0])
    in_right_bottom = (1150, imshape[0])
    in_left_top = (700,480)
    in_right_top = (650,480)
    vertices = np.array([[left_bottom, left_top, right_top, right_bottom, in_right_bottom,
                          in_left_top, in_right_top, in_left_bottom]], dtype=np.int32)
    # Masked area
    return region_of_interest(img, vertices)

In [229]:
def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(image, window_width, window_height, margin):
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
       # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))

    return window_centroids


def draw_window_centroids(img, window_width, window_height, margin):
    window_centroids = find_window_centroids(warped, window_width, window_height, margin)

    # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
 
    # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

    print(img.shape)
    print(output.shape)
    result = cv2.addWeighted(img, 1, output, 0.3, 0)
    plt.imshow(result)
    
    plt.title('window fitting results')
    plt.show()


In [230]:
original = cv2.imread("test_images/test5.jpg")

Canny_params = original, (50,150)
SobelX_params = original, 7, (10,255)
SobelY_params = original, 7, (10,255)
Sobel_params = original, True, True, SobelX_params, SobelY_params
Mag_params = original, 7, (10,30)
Dir_params = original, 7, (0.65, 1.05)
MagDir_params = original, True, True, Mag_params, Dir_params
HLS_params = original, (100,256)

thresholded = combine_threshold(original, Canny=False, Sobel=True, MagDir=True, HLS=True,
                               Canny_params=Canny_params, Sobel_params=Sobel_params, 
                                MagDir_params=MagDir_params, HLS_params=HLS_params)

masked = mask_image(thresholded)
M=get_perspective_transform()
warped = perspective_transform(masked,M)

#plt.subplot(121)
#plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))

#plt.subplot(122)
#plt.imshow(warped, cmap='gray')

#plt.show()

#plt.figure()
#histogram = np.sum(warped[warped.shape[0]//2:,:], axis=0)
#plt.plot(histogram)
#plt.show()

# window settings
window_width = 50 
window_height = 80 # Break image into 9 vertical layers since image height is 720
margin = 100 # How much to slide left and right for searching
draw_window_centroids(warped, window_width, window_height, margin)

(720, 1280)
(720, 1280, 3)


error: /build/opencv/src/opencv-3.2.0/modules/core/src/arithm.cpp:659: error: (-209) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function arithm_op
